To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.4.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
import json
jsonfig = """
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "https://followmetrios.com/l-endometriose-la-maladie-des-femmes/definition/", "output": "https://followmetrios.com/l-endometriose-la-maladie-des-femmes/definition/"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "EN 2024, LPG® s’engage auprès des femmes atteintes d’endométriose", "output": "IN 2024, LPG® committed to supporting women suffering from endometriosis"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "et crée un protocole spécifique pour soulager cette pathologie.", "output": "and created a specific protocol to alleviate this illness."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "L’endométriose n’est enseignée aux futurs médecins que depuis trois ans, d’où les raisons d’une errance de diagnostic jusque-là.", "output": "Endometriosis has only been taught to future doctors for the last three years, which explains why diagnoses have been erratic until now."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Depuis la reconnaissance de cette pathologie, les progrès sur le diagnostic sont considérables, mais qu’en est-il du suivi, de l’accompagnement des femmes lorsque le diagnostic tombe ?", "output": "There has been considerable progress in diagnosis since the condition was first recognised, but what about follow-up and support for women once they have been diagnosed?"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Une fois le diagnostic posé, les solutions apportées sont assez pauvres :", "output": "Once the diagnosis has been made, the solutions offered are fairly poor:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Traitement hormonal en continu destiné à supprimer les règles", "output": "Continuous hormone treatment to suppress menstruation"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "La chirurgie qui consiste à retirer les lésions mais nécessite parfois l’ablation d’une partie", "output": "ou de la totalité de l’utérus et des ovaires…"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Surgery, which consists of removing the lesions but sometimes requires the removal of part", "output": "or all of the uterus and ovaries"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Et les réponses se révèlent encore plus complexes quand les lésions sont disséminées sur plusieurs organes.", "output": "And the response is even more complex when the lesions are spread over several organs."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "En impasse thérapeutique, fragilisées, les femmes atteintes d’endométriose font face à une véritable détresse physique et psychologique.", "output": "Having reached an impasse in their treatment, women suffering from endometriosis face real physical and psychological distress."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "LA PRISE EN CHARGE KINÉSITHÉRAPIQUE :", "output": "PHYSIOTHERAPY TREATMENT:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "UNE SOLUTION QUI FAIT SES PREUVES", "output": "A TRIED AND TESTED SOLUTION"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Le kinésithérapeute met le patient au centre des préoccupations (santé, bien-être physiologique et psychologique, ressenti) dans le cadre d’une santé intégrative.", "output": "Physiotherapists place the patient at the centre of their concerns (health, physiological and psychological well-being, feelings, etc.) as part of an integrative approach to health."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Un accompagnement efficient dans son parcours santé, qui a un seul objectif : comprendre, améliorer la prise en charge et donc apporter une réponse ajustée.", "output": "They provide efficient support throughout your health care pathway, with a single objective: to understand and improve your care and therefore provide an appropriate response."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Depuis plus de 35 ans, le groupe LPG® est convaincu du rôle essentiel des masseurs-kinésithérapeutes dans la prise en charge de nombreuses pathologies.", "output": "For more than 35 years, the LPG® Group has been convinced of the essential role played by physiotherapists in the treatment of a wide range of conditions."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Pour les accompagner quotidiennement, les équipes LPG®, épaulées par un comité scientifique de recherche (Co.S.I.Re) ont développé des dispositifs paramédicaux naturels et non agressifs, à des fins thérapeutiques.", "output": "To support them each day, the LPG® teams, supported by a scientific research committee (Co.S.I.Re), have developed natural, non-aggressive paramedical devices for therapeutic purposes."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Des protocoles travaillés avec, pour seul but, d’aider les kinés dans leur pratique afin d’améliorer la vie de leurs patients.", "output": "Protocols developed with the sole aim of helping physiotherapists in their practice to improve the lives of their patients."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "L’endométriose est une maladie gynécologique chronique et inflammatoire qui se caractérise par la présence, hors de la cavité utérine, de tissu semblable à celui de la muqueuse de l’utérus (appelée endomètre).", "output": "Endometriosis is a chronic, inflammatory gynaecological disease characterised by the presence, outside the uterine cavity, of tissue similar to the lining of the uterus (known as the endometrium)."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Cette maladie ne se résume pas, comme on peut souvent le croire, à des règles plus douloureuses que la moyenne.", "output": "There is more to this condition than the common misconception that menstruation is more painful than average."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Les douleurs des femmes souffrant d'endométriose peuvent impacter non seulement la région abdominale (règles douloureuses : dysménorrhée, douleurs pendant les rapports sexuels (dyspareunie), douleurs pelviennes et lombaires, fatigue chronique, troubles digestifs et urinaires, infertilité), mais aussi le rachis et les membres inférieurs.", "output": "The pain experienced by women suffering from endometriosis can affect not only the abdominal region (painful periods: dysmenorrhoea, pain during sexual intercourse (dyspareunia), pelvic and lower back pain, chronic fatigue, digestive and urinary problems, infertility, etc.), but also the spine and lower limbs."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "La kinésithérapie aide ces patientes en agissant sur les :", "output": "Physiotherapy helps these patients by acting on:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Inflammations diffuses,", "output": "Diffuse inflammation"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Adhérences des fascias abdominaux et thoraciques,", "output": "Abdominal and thoracic fascia adhesions"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Fibroses autour du ligament utérin,", "output": "Fibrosis around the uterine ligament"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Dysfonctions du diaphragme,", "output": "Diaphragm dysfunction"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Tensions musculaires du petit bassin,", "output": "Muscle tension in the pelvis"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Paresthésies des nerfs pudendal,", "output": "Paraesthesia of the pudendal nerves"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Sciatiques ou cruraux,", "output": "Sciatica or thigh pain"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Postures en enroulement avec possibles pathologies rachidiennes.", "output": "Hunched posture with possible spinal conditions."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Les kinésithérapeutes formés traitent ces symptômes via des techniques comme la physiothérapie antalgique, les massages manuels, les étirements fasciaux et musculo-tendineux et l'activité physique adaptée.", "output": "Trained physiotherapists treat these symptoms using techniques such as analgesic physiotherapy, manual massage, fascial and musculotendinous stretching and adapted physical activity."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Il était, à ce titre, évident pour LPG® que les soins endermologie®, par l’action du massage mécanique, avaient un grand rôle à jouer dans l’amélioration de la condition de vie des patientes.", "output": "It was therefore obvious to LPG® that endermologie® treatments, through the action of mechanical massage, had a major role to play in improving patients' quality of life."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "NOUVEAU", "output": "NEW"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "PROTOCOLE ENDERMOLOGIE® MÉDICAL ENDOMÉTRIOSE", "output": "MEDICAL ENDERMOLOGIE® ENDOMETRIOSIS PROTOCOL"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Grâce au nouveau protocole endermologie®, la qualité de vie des patientes est considérablement améliorée par un mieux-être global :", "output": "The new endermologie®protocol, significantly improves patients' quality of life through an overall sense of well-being, by:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Soulage les douleurs", "output": "Relieving pain"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Relance la vascularisation", "output": "Boosting vascularisation"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Diminue les tensions musculaires", "output": "Reducing muscular tension"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Favorise la relaxation", "output": "Promoting relaxation"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Lutte contre la fibrose et les adhérences", "output": "Combatting fibrosis and adhesions"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "La mécanisation des tissus avec Cellu M6 Alliance® Médical est reconnue scientifiquement pour diminuer les douleurs (jusqu'à -50% en fonction du type de douleur*) en saturant les nocicepteurs (récepteurs sensoriels de la douleur) via les nombreuses stimulations mécaniques.", "output": "Tissue mechanisation with Cellu M6 Alliance® Medical is scientifically proven to reduce pain (by up to 50% depending on the type of pain*) by saturating the nociceptors (sensory pain receptors) via numerous mechanical stimulations."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Elle permet aussi de travailler sur la zone sans relancer de contractures réflexes.", "output": "It also makes it possible to work on the zone without triggering reflex contractions."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Cette action permet aussi de lutter tout en douceur contre les fibroses et les adhérences et de relancer la trophicité des zones dysfonctionnelles chroniques.", "output": "This action also gently combats fibrosis and adhesions and restores trophicity in chronically dysfunctional areas."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "*Observation clinique protocole endométriose", "output": "*Clinical observation of endometriosis protocol"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "En quatre étapes, le protocole endermologie® endométriose va agir sur les douleurs et la qualité de vie au quotidien.", "output": "In four steps, the endermologie® endometriosis protocol will combat pain and improve day-to-day quality of life ."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "ETAPE 1 :", "output": "STEP 1:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "BASSIN ET RACHIS - 15 MIN", "output": "PELVIS AND SPINE – 15 MIN"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Objectifs :", "output": "Objectives:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Effet parasympathique sur la colonne vertébrale", "output": "Parasympathetic effect on the spinal column"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Bassin et zone sacro lombaire : antalgie et amélioration de la circulation", "output": "Pelvis and sacro-lumbar area: pain relief and improved circulation"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Rachis dorso cervical : antalgie, anti-stress, anti-fatigue, mieux-être,", "output": "relaxation"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Dorsocervical spine: pain relief, anti-stress, anti-fatigue, well-being,", "output": "relaxation"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "ETAPEs 2 & 3 :", "output": "STEPS 2 & 3:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "DIAPHRAGME ET PILLIERS DU DIAPHRAGME - 5 MIN", "output": "DIAPHRAGM AND PILLARS OF THE DIAPHRAGM – 5 MIN"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "ETAPE 4 :", "output": "STEP 4:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "VENTRE, PLEXUS SOLAIRE ET STERNUM - 10 MIN", "output": "STOMACH, SOLAR PLEXUS AND STERNUM – 10 MIN"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Cartographie des points douloureux", "output": "Pain point mapping"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Objectifs :", "output": "Objectives:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Ventre : antalgie et amélioration du transit", "output": "Stomach: pain relief and improved digestive transit"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Plexus solaire et sternum : relaxation", "output": "Solar plexus and sternum: relaxation"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Ajout de manœuvres de spirale avec appui sur le plexus solaire et de rock sur les points douloureux.", "output": "Addition of spiral manoeuvres with pressure on the solar plexus and RockTape on painful points."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Objectifs :", "output": "Objectives:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Levée des tensions diaphragmatiques générées", "output": "Relief of diaphragmatic tensions caused"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "par les douleurs", "output": "by pain"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Antalgie, relaxation", "output": "Pain relief, relaxation"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "La douleur crée une tension permanente sur le diaphragme.", "output": "The pain creates permanent tension on the diaphragm."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Mieux respirer aide à gérer la douleur et favorise la relaxation.", "output": "Breathing better helps to manage pain and promotes relaxation."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Le protocole endométriose est pris en charge par la sécurité sociale.", "output": "The endometriosis protocol is covered by social security.."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Comme pour tout remboursement, une prescription de votre Chirurgien Gynécologue ou Médecin généraliste est nécessaire.", "output": "As with all reimbursements, a prescription from your Gynaecological Surgeon or General Practitioner is required."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Durée conseillée du traitement :", "output": "Recommended duration of treatment:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "12 semaines.", "output": "12 weeks."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Séances de +/-30 minutes, 1 à 2 séances/semaine", "output": "Sessions of +/-30 minutes, 1 to 2 sessions/week"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Cette recommandation fait suite à une observation clinique a été réalisée par plusieurs professionnels de santé.", "output": "This recommendation follows clinical observations made by several healthcare professionals."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Composition de l’équipe de l’observation clinique", "output": "Composition of the clinical observation team"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Prescripteurs", "output": "Prescribers"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Lieu d’exercice", "output": "Practice location"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Dr Chloé Lacoste", "output": "Dr Chloé Lacoste"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Hôpital St Joseph, Paris, 75", "output": "Hôpital St Joseph, Paris, 75"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Dr Emilie Piquet-Raynaud", "output": "Dr Emilie Piquet-Raynaud"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Clinique St Jean, Cagnes s/Mer, 06", "output": "Clinique St Jean, Cagnes s/Mer, 06"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Dr Frédéric Chiche", "output": "Dr Frédéric Chiche"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Hôpital Américain, Neuilly, 92", "output": "American Hospital, Neuilly, 92"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Dr Fabien Vidal", "output": "Dr Fabien Vidal"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Clinique de la Croix du Sud, Toulouse, 31", "output": "Clinique La Croix du Sud, Toulouse, 31"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Dr Horace Roman", "output": "Dr Horace Roman"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Clinique Tivoly, Bordeaux, 33", "output": "Clinique Tivoli, Bordeaux, 33"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Kinés référents", "output": "Specialist physiotherapists"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Lieu d’exercice", "output": "Practice location"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Julien Bendavid", "output": "Julien Bendavid"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Clichy, 92", "output": "Clichy, 92"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Sara Valabregue", "output": "Sara Valabregue"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Mouans-Sartoux, 06", "output": "Mouans-Sartoux, 06"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Nathalie Paradis", "output": "Nathalie Paradis"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Sophia-Antipolis, 06", "output": "Sophia-Antipolis, 06"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Marie Gozzi", "output": "Marie Gozzi"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Nice, 06", "output": "Nice, 06"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Florence Charpentier", "output": "Florence Charpentier"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Toulouse, 31", "output": "Toulouse, 31"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Sabrina Fajau", "output": "Sabrina Fajau"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Toulouse, 31", "output": "Toulouse, 31"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Dorothée Delecour", "output": "Dorothée Delecour"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Bordeaux, 33", "output": "Bordeaux, 33"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "LES RETOURS DES KINESITHERAPEUTES SUR LE PROTOCOLE", "output": "FEEDBACK ON THE PROTOCOL FROM PHYSIOTHERAPISTS"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "On cherche à activer le système nerveux automatique parasympathique, qui est le chef d’orchestre de la relaxation globale, de la diminution du stress et par la même occasion d’une partie des douleurs.", "output": "The aim is to activate the automatic parasympathetic nervous system, which is responsible for overall relaxation, reducing stress and, at the same time, minimising some of the pain."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "S’en suit un but défibrosant qui est l’essence de l’endermologie® medicale.", "output": "This is followed by a defibrosing effect which is the essence of medical endermologie®."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Ainsi les douleurs mécaniques dues à la perte de mobilité des tissus s’estompent séance après séance.", "output": "The mechanical pain caused by the loss of tissue mobility is therefore minimised with each session."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Sabrina Fajau @sab santé (Toulouse, 31)  Kinésithérapeute et Ostéopathe spécialisé en pelvi-périnéologie et en traitement de la peau et des cicatrices.", "output": "Sabrina Fajau @sab santé (Toulouse, 31)   physiotherapist and osteopath specialising in pelvic perineology and the treatment of skin and scars."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Elles me donnent le sentiment d’avoir repris en main leur vie.", "output": "I feel like they have taken back control of their lives."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Nathalie Paradis (Sophia-Antipolis, 06)", "output": "Nathalie Paradis (Sophia-Antipolis, 06)"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Les patientes ressentent un mieux-être, un effet relaxant et une diminution des douleurs immédiatement après la séance et dans les jours suivants, ce qui permet de voir les femmes 1 fois par semaine.", "output": "Patients feel a sense of well-being, a relaxing effect and a reduction in pain immediately after the session and in the days that follow, which means that women can be seen once a week."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Les cicatrices sont plus souples et moins adhérentes.", "output": "Scars are more supple, with fewer adhesions."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Certaines notent également un meilleur confort digestif.", "output": "Some also report improved digestive comfort."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Florence Charpentier (Toulouse, 31)", "output": "Florence Charpentier (Toulouse, 31)"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "- drainage des ganglions du plus inguinale très apprécié car cela stimule pour beaucoup le drainage de leurs jambes lourdes/bas ventre gorgé/psoas raide et remplis de toxines,", "output": "- drainage of the most inguinal lymph nodes, which is greatly appreciated as it stimulates the drainage of their heavy legs/bloated lower abdomen/stiff, toxin-filled pelvis."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "- drainage bas ventre et travail sur ovaire en spirale ou twist très apprécié lors des règles et bassin engorgé,", "output": "- drainage of the lower abdomen and work on the ovary in a spiral or twisting motion, which is greatly appreciated during menstruation and when the pelvis is congested."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "- passage sur le diaphragme apprécié pour celle qui ont une ptose abdominale ou restriction respiratoire lors de crise de crampe ou encore les attaches des piliers du diaphragme ainsi que ses insertions tout le long des côtes : détente et gain d’amplitude à la respiration surtout après des phases de replis post crise.", "output": "- passage over the diaphragm, which is appreciated by those who have abdominal ptosis or respiratory restriction during cramp attacks, or adhesions on the pillars of the diaphragm and along the ribs: relaxation and ability to breathe deeper especially once the period of cramping has passed."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "- passage postérieur très intéressant sur le sacrum pour désengorger et détendre ses insertions et détoxification des lombes sur les passages lombaires sensibles mais néanmoins nécessaires pour beaucoup des patientes.", "output": "- posterior passage on the sacrum is very useful for relieving congestion and easing its insertions, and detoxification of the lumbar vertebrae on the lumbar passages can be sensitive but is, nonetheless, necessary for many patients."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Anne-Sophie Miloszyk (Bordeaux, 33)", "output": "Anne-Sophie Miloszyk (Bordeaux, 33)"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "ACTU SOCIÉTÉ:", "output": "GROUP NEWS:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "La 20ème semaine européenne de prévention et d’information sur l’endométriose aura lieu du 4 au 10 mars 2024.", "output": "The 20th European Endometriosis Prevention and Information Week will take place from 4 to 10 March 2024."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "LPG® ENGAGÉ DANS LA FONDATION POUR LA RECHERCHE SUR L’ENDOMÉTRIOSE (FRE)", "output": "LPG® IS COMMITTED TO THE FONDATION POUR LA RECHERCHE SUR L’ENDOMÉTRIOSE (FRE, ENDOMETRIOSIS RESEARCH FOUNDATION)"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "En 2022, LPG® a fait un premier don à la Fondation.", "output": "In 2022, LPG® made its first donation to the foundation."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "En 2023, LPG® a souhaité s’engager plus loin en faveur de la recherche sur l’endométriose en signant une convention de produit-partage tout en renouvelant son mécénat financier.", "output": "In 2023, LPG® wanted to make a greater commitment to endometriosis research by signing a product-sharing agreement and renewing its financial sponsorship."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "LPG® a reversé 1€ par produit vendu lors d’une opération de communication menée au printemps dernier et va réitérer l’opération sur 2024.", "output": "LPG® donated €1 per product sold during a promotional campaign last spring and will be doing this again in 2024."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "En contrepartie de son mécénat, La Fondation a accompagné LPG® dans la création d’un protocole dédié aux soins apportés par les kinésithérapeutes aux patientes souffrant d’endométriose.", "output": "In return for its sponsorship, the foundation supported LPG® in the creation of a protocol dedicated to the care provided by physiotherapists to patients suffering from endometriosis."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Les masseurs-kinésithérapeutes spécialisés dans les différentes pathologies traitées avec les technologies LPG® restent à votre disposition.", "output": "Physiotherapists specialising in the various conditions treated with LPG® technologies are at your disposal."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Contactez-nous pour toutes demandes d’interviews.", "output": "Contact us for interview requests."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Chapitre 1 :", "output": "Chapter 1:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Prise en charge kinésithérapique en sénologie :", "output": "Physiotherapy management in senology:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Dorothée Delecour, Masseur-Kinésithérapeute à Bordeaux.", "output": "Dorothée Delecour, masseur-physiotherapist in Bordeaux."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Chapitre 2 :", "output": "Chapter 2:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Prise en charge kinésithérapique de la sclérose en plaques :", "output": "Physiotherapy management of multiple sclerosis:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Caroline Peccabin, Kinésithérapeute à la Tour de Gassies à Bruges.", "output": "Caroline Peccabin, Physiotherapist at Tour de Gassies in Bruges."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Chapitre 3 :", "output": "Chapter 3:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Prise en charge kinésithérapique de l’endométriose :", "output": "Physiotherapy management of endometriosis:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Sabrina Fajau @sabsante, Masseur-Kinésithérapeute à Toulouse.", "output": "Sabrina Fajau @sabsante, masseur-physiotherapist in Toulouse."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Chapitre 4 :", "output": "Chapter 4:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Prise en charge kinésithérapique des lombalgies et maux de dos :", "output": "Physiotherapy management of lumbago and back pain:."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Julien Drouot, Masseur-Kinésithérapeute à Nancy.", "output": "Julien Drouot, masseur-physiotherapist in Nancy."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Pour retrouver toutes les informations sur ce sujet :", "output": "For more information on this subject:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "lpgmedical.com", "output": "lpgmedical.com"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Les dates clefs de l’histoire de l’endométriose*", "output": "Key dates in the history of endometriosis*"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Égypte Antique (- 1855 av. J.C.)", "output": "Ancient Egypt (1855 BC):"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": ": des symptômes pouvant évoquer l’endométriose sont décrits sur papyrus.", "output": "symptoms suggestive of endometriosis are described on papyrus."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Antiquité grecque (- 500 à - 300 avant JC) : des médecins décrivent des troubles des règles liés à l’utérus.", "output": "Ancient Greece (500 BC to 300 BC): doctors described menstrual problems linked to the uterus."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Moyen-Âge et Renaissance (776 à 1600) : les symptômes évoquant l’endométriose sont associés à la possession démoniaque : l’hystérie !", "output": "Middle Ages and Renaissance (776 to 1600): symptoms suggestive of endometriosis were associated with demonic possession i.e. hysteria!"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "XIXe siècle : étude des lésions et des symptômes qui aujourd’hui évoquent une endométriose.", "output": "19th century: study of lesions and symptoms which today suggest endometriosis."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Grâce à l'essor de la médecine moderne et notamment à la contribution très significative de l'école anatomopathologique française, couronnée par cette déclaration avant-gardiste et toujours d'actualité d'Armand Trousseau :", "output": "Thanks to the development of modern medicine and, in particular, to the very significant contribution of the French anatomopathological school, crowned by Armand Trousseau's avant-garde and still relevant declaration:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "À partir de là, les définitions de la maladie se précisent petit à petit, jusqu’à l’apparition en 1927 du nom que nous connaissons tous : l’endométriose posé par J.A.", "output": "From then on, definitions of the condition gradually became more precise, until in 1927 the name we all know appeared: endometriosis, coined by J.A."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Sampson.", "output": "Sampson."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Il développe la première théorie explicative de la maladie, toujours valide : le reflux des règles par les trompes dans la cavité pelvienne.", "output": "He developed the first theory to explain the disease, which is still valid: the reflux of menstrual periods through the fallopian tubes into the pelvic cavity."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Jusqu’à la fin du XXème siècle, les interprétations autrefois psychiques laissent peu à peu place à la définition actuelle de l’endométriose, recentrée sur les aspects physiologiques de la maladie.", "output": "Up until the end of the 20th century, psychological interpretations gradually gave way to the current definition of endometriosis, which focuses on the physiological aspects of the condition."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "2020 : l’endométriose rentre dans le programme de médecine de second cycle.", "output": "2020: endometriosis is included in the second cycle medical curriculum."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "* lyv.app - startup dont la mission est de révolutionner la prise en charge de l’endométriose", "output": "*lyv.app – a start-up whose mission is to revolutionise the way endometriosis is treated"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "CONTACT PRESSE :", "output": "PRESS CONTACT:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "AGENCE WELL COM MEDIA", "output": "WELL COM MEDIA AGENCY"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "maeva.batailles@wellcommedia.fr", "output": "maeva.batailles@wellcommedia.fr"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "& caroline.falce@wellcommedia.fr", "output": "& caroline.falce@wellcommedia.fr"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "lpg-group.com", "output": "lpg-group.com"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Pour trouver les kinésithérapeutes équipés :", "output": "To find qualified physiotherapists:"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "LPG Medical > Nos centres > Trouver un praticien", "output": "LPG Medical > Our centres > Find a practitioner"}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "2208 - FR © LPG Systems 2024 - S.A.S. au capital de 6 404 850€ - Siren RCS 335.183.836 Romans - Toute reproduction est strictement interdite.", "output": "2208 – FR © LPG Systems 2024 - S.A.S. with capital of €6,404,850 – Siren RCS 335.183.836 Romans – Reproduction is strictly prohibited."}
{"instruction": "You are a helpful translator assistant who only answer with the translation in English of what it's been given, when you see a word that's an abreviation, like ADV, keep it as it is.", "input": "Photos non contractuelles.", "output": "Non-contractual photos."}
"""

examples = [json.loads(line) for line in jsonfig.strip().splitlines()]


In [5]:
from datasets import Dataset
dataset = Dataset.from_list(examples)


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Alpaca.ipynb)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass
dataset = Dataset.from_list(examples)
dataset=dataset.map(formatting_prompts_func, batched=True)
#from datasets import load_dataset
#dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
#dataset = dataset.map(formatting_prompts_func, batched = True,)"

Map:   0%|          | 0/163 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 40,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/163 [00:00<?, ? examples/s]

In [11]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.137 GB of memory reserved.


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 163 | Num Epochs = 2 | Total steps = 40
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,1.521100
2,1.382800
3,1.343200
4,0.943100
5,0.826600
6,0.917200
7,0.858500
8,0.719700
9,0.630100
10,0.694300


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

462.7198 seconds used for training.
7.71 minutes used for training.
Peak reserved memory = 7.922 GB.
Peak reserved memory for training = 1.938 GB.
Peak reserved memory % of max memory = 53.716 %.
Peak reserved memory for training % of max memory = 13.141 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Unsloth_Studio.ipynb)**

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
13, 21, 34, 55, 89, 144<|end_of_text|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is a famous tall tower in Paris?

### Input:


### Response:
One of the most famous and iconic tall towers in Paris is the Eiffel Tower. Standing at 324 meters (1,063 feet) tall, this wrought iron tower is a symbol of the city and a must-see attraction for tourists from all over the world.<|end_of_text|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [13]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.75 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 47%|████▋     | 15/32 [00:01<00:01, 12.99it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [02:01<00:00,  3.78s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /content/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf:

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
